In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import os
import xarray as xr

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import funcs_general as fg

In [ ]:
# Load observational metrics

obsdsetlist = ['PRISM', 'CONUS404', 'Livneh', 'NLDAS', 'GMET','nClimGrid','gridMET']
nobs = len(obsdsetlist)
diri = '/glade/work/nlybarger/downscaling_metrics/obs/'

obsmaps = {}
for obs in obsdsetlist:
    f =  f'{diri}{obs}.1981-2016.ds.conus.metric.maps.nc'
    obsmaps[obs] = xr.open_dataset(f)
varlist = list(obsmaps['GMET'].variables)
for var in ['lat','lon','time']:
    if var in varlist:
        varlist.remove(var)

vmin = np.zeros(len(varlist))
vmax = np.zeros(len(varlist))
for varind,var in enumerate(varlist):
    vmin[varind] = min([obsmaps[obs][var].min() for obs in obsdsetlist])
    vmax[varind] = max([obsmaps[obs][var].max() for obs in obsdsetlist])
nvar = len(varlist)
latty = obsmaps['GMET']['lat'].copy()
lonny = obsmaps['GMET']['lon'].copy()
nlat = len(latty)
nlon = len(lonny)


In [ ]:
## Figure 3

for var in ['t90']:
    varind = varlist.index(var)
    fig, axes = plt.subplots(nobs, nobs, figsize=(22, 12), subplot_kw={'projection': ccrs.PlateCarree()})
    axs = axes.flat

    for irow, obsi in enumerate(obsdsetlist):
        for icol, obsj in enumerate(obsdsetlist):
            pltind = irow * nobs + icol

            if icol >= irow:
                axs[pltind].remove()
            else:
                ax = axs[pltind]
                ax.set_extent([lonny.min(), lonny.max(), latty.min(), latty.max()], crs=ccrs.PlateCarree())
                ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
                ax.add_feature(cfeature.BORDERS, linewidth=0.5)
                ax.add_feature(cfeature.STATES, linewidth=0.3, edgecolor='gray')

                # Plot the data
                im = ax.pcolormesh(
                    lonny, latty, obsmaps[obsi][var] - obsmaps[obsj][var],
                    vmin=-7, vmax=7, cmap='seismic', transform=ccrs.PlateCarree()
                )
                ax.set_title(f'{obsi} - {obsj}', fontsize=16)

    # Add a colorbar
    cbar_ax = fig.add_axes([0.25, 0.8, 0.3, 0.02])  # Adjust position and size of the colorbar
    cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
    cbar.ax.tick_params(labelsize=12)
    # cbar.set_label(var, fontsize=14)

    # Set figure properties
    fig.set_facecolor('white')
    # plt.suptitle(var, y=0.95, x=0.3, fontsize=24)
    plt.tight_layout(rect=[0, 0, 0.9, 1])  # Adjust layout to make space for the colorbar

    # Save the figure
    # plt.savefig(f'/glade/work/nlybarger/images/downscaling/paper_plots/obs.diffs.{var}.png', bbox_inches='tight', dpi=600)
    # plt.close()

In [ ]:
# Figure 2

mod = 'CanESM2'
methods = ['ICAR','ICARwest','GARD_r2','GARD_r3','LOCA_8th','MACA','NASA-NEX']
nmeth = len(methods)
diri = '/glade/work/nlybarger/downscaling_metrics/cmip5/'

dsmaps = {}
for meth in methods:
    fili = f'{mod}.{meth}.hist.1981-2016.ds.conus.metric.maps.nc'
    dsmaps[meth] = xr.open_dataset(diri+fili)
    dsmaps[meth]['ttrend'] = dsmaps[meth]['ttrend']
    dsmaps[meth]['ptrend'] = dsmaps[meth]['ptrend']

varlist = list(dsmaps[methods[0]].variables)
for var in ['lat','lon','time']:
    if var in varlist:
        varlist.remove(var)

vmin = np.zeros(len(varlist))
vmax = np.zeros(len(varlist))
for varind,var in enumerate(varlist):
    vmin[varind] = min([dsmaps[meth][var].min() for meth in methods])
    vmax[varind] = max([dsmaps[meth][var].max() for meth in methods])
nvar = len(varlist)
latty = dsmaps[methods[0]]['lat'].copy()
lonny = dsmaps[methods[0]]['lon'].copy()
nlat = len(latty)
nlon = len(lonny)

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Define variables to plot
pltvars = ['n34pr', 'ptrend', 'ttrend', 'pr90', 't90']
varplts = ['Nino3.4 Pr', 'Pr-Trend', 'T-Trend', 'Pr90', 'T90']
vmin = [-0.8, -100, -0.2, 0, 10]
vmax = [0.8, 100, 1., 12, 30]
npltv = len(pltvars)
cmaps = ['seismic', 'BrBG', 'turbo', 'BrBG', 'turbo']
methplt = {'ICAR': 'ICARv1', 'ICARwest': 'ICARv2', 'GARD_r2': 'GARD-puv', 'GARD_r3': 'GARD-quv', 
           'LOCA_8th': 'LOCA', 'MACA': 'MACA', 'NASA-NEX': 'NASA-NEX'}
# Create the figure and axes
fig, axes = plt.subplots(
    nmeth+1, npltv, figsize=(19, 17),
    subplot_kw={'projection': ccrs.PlateCarree()}
)
axs = axes.flat

# Loop through variables and methods
for ivar, var in enumerate(pltvars):
    # varind = varlist.index(var)

    pltvmax = vmax[ivar]
    pltvmin = vmin[ivar]

    for imeth, meth in enumerate(methods):
        pltind = imeth * npltv + ivar
        ax = axs[pltind]
        ax.set_extent([lonny.min(), lonny.max(), latty.min(), latty.max()], crs=ccrs.PlateCarree())
        ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
        ax.add_feature(cfeature.BORDERS, linewidth=0.5)
        ax.add_feature(cfeature.STATES, linewidth=0.3, edgecolor='gray')

        # Plot the data
        im = ax.pcolormesh(
            lonny, latty, dsmaps[meth][var],
            vmin=pltvmin, vmax=pltvmax, cmap=cmaps[ivar],
            transform=ccrs.PlateCarree()
        )
        # ax.set_title(f'{methplt[meth]}', fontsize=16)#, fontdict={'fontname': 'Verdana'})
        if ivar == 0:
            ax.text(
                -0.1, 0.5, methplt[meth],  # Adjust position as needed
                fontsize=18,
                ha='center', va='center',
                rotation='vertical',
                transform=ax.transAxes
            )
        if imeth == 0:
            ax.set_title(varplts[ivar], fontsize=18)

    imeth += 1
    pltind = imeth * npltv + ivar
    ax = axs[pltind]
    for spine in ax.spines.values():
        spine.set_linewidth(4)
    ax.set_extent([lonny.min(), lonny.max(), latty.min(), latty.max()], crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.add_feature(cfeature.STATES, linewidth=0.3, edgecolor='gray')
    
    # Plot the data
    im = ax.pcolormesh(
        lonny, latty, obsmaps['PRISM'][var],
        vmin=pltvmin, vmax=pltvmax, cmap=cmaps[ivar],
        transform=ccrs.PlateCarree())
    
    if ivar == 0:
        ax.text(
            -0.1, 0.5, 'PRISM',  # Adjust position as needed
            fontsize=18,
            ha='center', va='center',
            rotation='vertical',
            transform=ax.transAxes)

    cbar_ax = fig.add_axes([0.04 + ivar * 0.195, 0.08, 0.16, 0.02])  # Adjust position and size for alignment
    cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
    cbar.ax.tick_params(labelsize=14)


# Set figure properties
fig.set_facecolor('white')
plt.tight_layout(rect=[0, 0.1, 1, 0.95])

# Uncomment to save the figure
# plt.savefig('/glade/work/nlybarger/images/downscaling/paper_plots/CanESM2_multivar_dsmaps.png', bbox_inches='tight', dpi=600)
# plt.show()